# Fdem Data Point Class

## Fdem Data contains entire data sets
## Fdem Data Points can forward model and evaluate themselves

### FdemData is an extension to the [Data](Data.ipynb) Class

##### Back to [Main](../../PackageInfo.ipynb)

In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

from os.path import join
import numpy as np
import h5py
import matplotlib.pyplot as plt
from geobipy import hdfRead
from geobipy import FdemData
from geobipy import FdemDataPoint
from geobipy import Model1D
from geobipy import StatArray

In [ ]:
# The data file name
dataFile=join('supplementary','Data','Resolve2.txt')
# The EM system file name
systemFile=join('supplementary','Data','FdemSystem2.stm')

## Initialize and read an EM data set

In [ ]:
D=FdemData()
D.read(dataFile,systemFile)

## Summarize the Data

In [ ]:
D.summary()

## Grab a measurement from the data set

In [ ]:
P=D.getDataPoint(0)
P.sys.summary()
P.summary()
plt.figure()
P.plot()

## We can forward model the EM response of a 1D layered earth <a href="../Model/Model1D.ipynb">Model1D</a>

In [ ]:
par = StatArray(np.linspace(0.01, 0.1, 10), "Conductivity", "$\\frac{S}{m}$")
thk = StatArray(np.ones(10) * 10.0)
mod = Model1D(nCells = 10, parameters=par, thickness=thk)
mod.summary()
#plt.figure()
#mod.plotBlocks()

## Compute and plot the data from the model

In [ ]:
P.forward(mod)
plt.figure()
P.plot()
P.plotPredicted()

## The errors are set to zero right now, so lets change that

In [ ]:
# Set the Prior
addErrors = StatArray(np.full(2*P.sys.nFreq, 10.0))
P.p.setPrior('MVNormalLog', addErrors, addErrors)
P.updateErrors(1, P.s[:], 0.1, addErrors[:])

## With forward modelling, we can solve for the best fitting halfspace model

In [ ]:
HSconductivity=P.FindBestHalfSpace()
print('Best half space conductivity is ', HSconductivity, ' $S/m$')
plt.figure()
P.plot()
P.plotPredicted()

## Compute the misfit between observed and predicted data

In [ ]:
print(P.dataMisfit())

## Plot the misfits for a range of half space conductivities

In [ ]:
plt.figure()
P.plotHalfSpaceResponses(-6.0,4.0,200)

## Compute the sensitivity matrix for a given model

In [ ]:
J = P.sensitivity(mod)
plt.figure()
J.pcolor()

## We can save the FdemDataPoint to a HDF file

In [ ]:
with h5py.File('FdemDataPoint.h5','w') as hf:
    P.writeHdf(hf, 'fdp')

## And then read it in

In [ ]:
P1=hdfRead.readKeyFromFiles('FdemDataPoint.h5','/','fdp')

In [ ]:
P1.summary()